In [14]:
import polars as pl
from polars import col as c
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp
from distflow_algorithm import DistFlow
import time 
import pandapower as pp
import pandapower.networks as pn
from polars_function import get_transfo_admittance, get_transfo_impedance, get_transfo_conductance, get_transfo_imaginary_component

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, generate_log
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix,generate_bfs_tree_with_edge_data
import graphblas as gb


In [15]:
log = generate_log(name=__name__)

line_data = pl.read_csv("../data/lineData.csv").with_columns(
    pl.lit("branch").alias("type"),
    pl.lit(0.0).alias("g_pu"),
    pl.lit(1.0).alias("n_transfo"),
)
ext_grid_nb = 0
nx_grid: nx.Graph = nx.Graph()
_ = line_data\
.with_columns(
    pl.struct(pl.all()).pipe(generate_nx_edge, nx_graph=nx_grid)
)
if not nx.is_tree(nx_grid):
    log.error("The graph is not a tree")
else:
    nx_tree_grid : nx.DiGraph = generate_bfs_tree_with_edge_data(nx_grid, ext_grid_nb)


In [31]:
nx_tree_grid.number_of_nodes()

58

In [63]:
n_nodes = nx_tree_grid.number_of_nodes()

coords, ancestor = list(zip(*nx.all_pairs_lowest_common_ancestor(nx_tree_grid)))
x, y = list(zip(*coords))
r_mapping = nx.shortest_path_length(nx_tree_grid, source=0, weight="r_pu")
x_mapping = nx.shortest_path_length(nx_tree_grid, source=0, weight="x_pu")

r_val = list(map(lambda x: -2*r_mapping[x], ancestor))
x_val = list(map(lambda x: -2*x_mapping[x], ancestor))


h_Vnode_Pload = gb.Matrix.from_coo(x, y, r_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[1:, 1:]
h_Vnode_Pload = gb.select.offdiag(h_Vnode_Pload).T + h_Vnode_Pload

h_Vnode_Qload = gb.Matrix.from_coo(x, y, r_val, nrows=n_nodes, ncols=n_nodes).select("!=", 0)[1:, 1:]
h_Vnode_Qload = gb.select.offdiag(h_Vnode_Qload).T + h_Vnode_Qload

h_Pflow_Pload = generate_shortest_path_length_matrix(nx_grid = nx_tree_grid, forced_weight=1)[1:, 1:]

In [74]:
line_data

line_id,u_of_edge,v_of_edge,r_pu,x_pu,b_pu,rateA,rateB,type,g_pu,n_transfo
i64,i64,i64,f64,f64,f64,f64,i64,str,f64,f64
1,0,1,0.19375,0.11297,7.9168e-8,0.024942,2,"""branch""",0.0,1.0
2,0,2,2.0469,1.1462,8.9632e-7,0.026327,2,"""branch""",0.0,1.0
3,0,3,0.0,0.0,0.0,1.0,3,"""branch""",0.0,1.0
4,1,4,3.03,0.0911,2.2696e-8,0.0030484,2,"""branch""",0.0,1.0
5,1,5,0.0775,0.045187,3.1667e-8,0.024942,2,"""branch""",0.0,1.0
…,…,…,…,…,…,…,…,…,…,…
53,52,53,3.7875,0.11388,2.8370e-8,0.0030484,2,"""branch""",0.0,1.0
54,52,54,0.08525,0.049706,3.4834e-8,0.024942,2,"""branch""",0.0,1.0
55,54,55,0.05425,0.031631,2.2167e-8,0.024942,2,"""branch""",0.0,1.0


In [75]:

line_data.group_by("u_of_edge").agg(c("b_pu").sum()).sort("u_of_edge")



u_of_edge,b_pu
i64,f64
0,9.7549e-7
1,5.4363e-8
2,3.1186e-7
3,0.0
5,5.6871e-8
…,…
48,1.8096e-8
49,6.1455e-8
52,6.3204e-8
